In [1]:
# Packages 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

## Deep SAD(Deep semi-supervised anomaly detection)
본 노트북의 코드는 논문 *"Deep Semi-Supervised Anomlay Detection(Lukas Ruff et al. ICLR 2020)"* 의 이론 및 알고리즘을 바탕으로 작성되었다. pytorch로 작성된 해당 논문의 코드(https://github.com/lukasruff/Deep-SAD-PyTorch/blob/master/src/DeepSAD.py)와 이를 텐서플로로 작성한 코드 (https://github.com/JungWoo-Chae/DeepSAD)를 바탕으로 본 competition의 dataset에 맞게끔 작성하였다.

### DeepSVDD
DeepSAD 알고리즘은 딥러닝을 이용한 anomaly detection 알고리즘인 Deep SVDD(Ruff et al., 2018)을 Semi-supervised learning, 즉 본 문제와 같이 labeled sample이 추가로 주어진 상황으로 확장시킨 알고리즘이다 여기서 label은 normal data의 경우 $+1$, anomaly에 대해서는 $-1$의 값을 갖도록 주어진다. 주어진 $n$개의 unlabeled training samples $x_1,\ldots,x_n\in\mathcal X$에서 Deep SVDD의 목적함수는 다음과 같다.
$$
\min_W{1\over n}\sum_{i=1}^n\Vert\phi(x_i;W)-c\Vert^2 + {\lambda\over 2}\sum_{l=1}^L\Vert W^l\Vert_F^2,\;\;\lambda>0
$$
즉, hypersphere의 중심점인 상수벡터 c를 중심으로 mean squared distance를 최소화하는 알고리즘이므로, 이는 곧 데이터들의 공통된 특성을 추출하는 과정으로 볼 수 있다. 최적화 과정은 SGD를 이용한 backpropagation으로 이루어지며 trained model에 대한 anomaly score는
$$
s(x) = \Vert \phi(x;W)-c\Vert
$$
로 주어진다.

### DeepSAD
반면, DeepSAD의 경우 semi-supervised learning 문제이므로 labeled data를 최적화 식에 반영하여 다음과 같은 목적함수를 취한다.
$$
\min_W{1\over n+m}\sum_{i=1}^n\Vert\phi(x_i;W)-c\Vert^2 +{\eta\over n+m}\sum_{j=1}^m(\Vert\phi(\tilde x_j;W)-c\Vert^2)^{\tilde y_j} +{\lambda\over 2}\sum_{l=1}^L\Vert W^l\Vert_F^2,\;\
$$

여기서 두번째 항의 샘플
$$
(\tilde x_1,\tilde y_1),\ldots,(\tilde x_m, \tilde y_m)
$$
이 labeled data에 해당하며, 상수 $\eta$는 이 labeled data의 norm을 얼마나 반영할 것인지 정하는 상수이다. 즉, $\eta>1$이면 unlabeled data보다 labeled data를 더 많이 반영하겠다는 설정이 된다.

### Model Configuration

In [4]:
# Configuration
config = {
    'randomstate' : 123,
    'test_size' : 0.3,
    'batch_size' : 32,
    'eta' : 1.5,  ## eta of second term
    'pretrain' : True, # Pretraining(autoencoder)
    'ae_lr' : 0.01, # learning rate of autoencoder
    'ae_epochs' : 10,
    'lr' : 0.1, # learning rate of model
    'epochs' : 10 
}

### Data Load

In [5]:
# Data load
from dataset.main import load_dataset

train_data, test_data = load_dataset(config)
print(f"Train : {len(list(train_data.as_numpy_iterator()))}\nTest : {len(list(test_data.as_numpy_iterator()))}")

Train : 99612
Test : 8539


### Model Training

In [6]:
# Logging
from utils import set_memory_growth
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
set_memory_growth()

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)

In [7]:
# Model Training
from DeepSAD import DeepSAD

model = DeepSAD(config)
model.set_network()

In [8]:
# Pretrain
if config['pretrain'] :
    model.pretrain(train_data, lr=config['ae_lr'], epochs=config['ae_epochs'])

Start Pretraining


KeyboardInterrupt: 

In [ ]:
# Train

# model.train(train_data, lr=config['lr'], epochs=config['epochsz'])


In [ ]:
# Test

In [19]:
i = 0
for inputs, labels in train_data:
    print(inputs)
    print(labels)
    i += 1
    if i > 5:
        break

tf.Tensor(
[ 1.23308366  0.37327848  0.44541781  0.76873926 -0.51756268 -1.25616424
  0.10402938 -0.23317549 -0.01227257 -0.28758426  0.25054862  0.42124156
  0.31692062 -0.29737707  1.05390591  0.37659058  0.10472026 -0.42361192
 -0.32226681 -0.08143418 -0.26436408 -0.75789729  0.16768024  0.67555997
  0.17755208  0.07369328 -0.02171477  0.0344787  -0.27974569 -0.96154795], shape=(30,), dtype=float64)
tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(
[-3.81784667  2.19114879 -0.01434616  0.26997635 -0.09356548  1.46306027
 -3.01740883 -7.69396934 -0.67148272 -1.26827173  0.64440138  1.72800768
 -1.00259434  1.16811505 -1.66062294  0.20100846  0.11044882 -0.36141366
 -1.17865883  1.05257409 -3.76718601  1.45985256  0.19487667 -0.01522933
 -0.24077814 -0.69534668 -1.17752207  0.23126055  0.39558443 -0.29683149], shape=(30,), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor(
[ 1.62382906 -0.06445982 -0.50910375  3.41457061  1.02385274  2.67821693
 -0.53593396  0.76310976